# Setup

## Install & Import

In [ ]:
!conda install -y gdown

In [2]:
%%capture
!pip install python-terrier -q
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git -q
!pip install -U sentence-transformers -q
!pip install --upgrade gensim
!pip install rake-nltk

In [3]:
%%capture
!pip install POT==0.4.0
from ot import emd2

In [ ]:
!sudo apt-get install -y openjdk-11-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
from IPython.display import display
from xml.dom.minidom import parse, parseString
from pyterrier.measures import *

from sentence_transformers import CrossEncoder

from gensim.models import Word2Vec, FastText

from rake_nltk import Rake

from scipy import stats
from scipy.spatial import distance
from scipy.spatial.distance import cosine

from sklearn import preprocessing
from sklearn.model_selection import KFold

import xml.etree.ElementTree as et
import pickle
import random
import pyterrier as pt
import pandas as pd
import numpy as np
import xgboost as xgb
import torch
import json
import os
import re
import math
import nltk
nltk.download('punkt')

if not pt.started():
    pt.init(version='snapshot')

In [6]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
!gdown 1qKm6yxQ2KzSiGkNaJSYgxV4nC_MGPUka # nostops_data_23-03-2024
!gdown 1cfgOF6kP8brxI_dtMTwhMEBGwgWBHjjV # nostops_queries_23-03-2024
!gdown 1PrLtsWEKk5HVd0gqPet93T8XLoIx0S_o # nostops_qrels_23-03-2024

!gdown 1eJxAmV1o4bT6VdE1sqmaVRLTuK1zn-Qb # stopwords.csv

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1QSRWJuEdODVFPsm_GxdLxQPGkaVfc-N1 # EMBEDDING

In [9]:
stops = pd.read_csv('stopwords.csv', header=None)[0].values.tolist()

In [80]:
data = pd.read_pickle("nostops_data_23-03-24.pickle")
queries = pd.read_pickle("nostops_queries_23-03-24.pickle")
qrels = pd.read_pickle("nostops_qrels_23-03-24.pickle")

In [81]:
t5_query = 'EMBEDDING/t5_query_26-03-24.pickle'
t5_doc = 'EMBEDDING/t5_data_26-03-24.pickle'
pminilm_query = 'EMBEDDING/pminilm_query_26-03-24.pickle'
pminilm_doc = 'EMBEDDING/pminilm_data_26-03-24.pickle'
bert_query = 'EMBEDDING/bert_query_26-03-24.pickle'
bert_doc = 'EMBEDDING/bert_data_26-03-24.pickle'

with open(t5_query, 'rb') as handle:
    t5_query_embeddings = pickle.load(handle)
with open(t5_doc, 'rb') as handle:
    t5_doc_embeddings = pickle.load(handle)
with open(pminilm_query, 'rb') as handle:
    pminilm_query_embeddings = pickle.load(handle)
with open(pminilm_doc, 'rb') as handle:
    pminilm_doc_embeddings = pickle.load(handle)
with open(bert_query, 'rb') as handle:
    bert_query_embeddings = pickle.load(handle)
with open(bert_doc, 'rb') as handle:
    bert_doc_embeddings = pickle.load(handle)

In [82]:
data.head()

,docno,keluhan,keluhan_preprocessed
0,DS-1,Mengapa Keringat Badan Sangat Berlebihan?. sel...,keringat badan selamat malam dokter fathurrosi...
1,DS-2,Mengapa Lengan Atas Sakit Walaupun Tidak Melak...,lengan sakit aktivitas siang dokter 4 lengan s...
2,DS-3,Berapa Batas Usia Kandungan Untuk Melakukan US...,batas usia kandungan usg batas usia kandungan ...
3,DS-4,Apakah Katarak Pada Bayi Harus Disembuhkan Den...,katarak bayi disembuhkan operasi selamat pagi ...
4,DS-5,Mengapa Badan Panas dan Ngilu Selama Berhari-h...,badan panas ngilu berhari nama ferini intan lu...


In [83]:
data = data.rename(columns = {"keluhan": "keluhan_raw", "keluhan_preprocessed": "keluhan"})
data.head()

,docno,keluhan_raw,keluhan
0,DS-1,Mengapa Keringat Badan Sangat Berlebihan?. sel...,keringat badan selamat malam dokter fathurrosi...
1,DS-2,Mengapa Lengan Atas Sakit Walaupun Tidak Melak...,lengan sakit aktivitas siang dokter 4 lengan s...
2,DS-3,Berapa Batas Usia Kandungan Untuk Melakukan US...,batas usia kandungan usg batas usia kandungan ...
3,DS-4,Apakah Katarak Pada Bayi Harus Disembuhkan Den...,katarak bayi disembuhkan operasi selamat pagi ...
4,DS-5,Mengapa Badan Panas dan Ngilu Selama Berhari-h...,badan panas ngilu berhari nama ferini intan lu...


In [84]:
queries.head()

,qid,query,query_preprocessed
0,Q1,"Cara mengatasi pusing, lemas dan hidung berair...",mengatasi pusing lemas hidung berair malam dok...
174,Q2,Telinga berdengung sebelah kiri. Selamat pagi ...,telinga berdengung sebelah kiri selamat pagi d...
308,Q3,Benjolan di sekitar kelamin. Sore dok..Maaf do...,benjolan kelamin sore dok maaf dok 4 mengalami...
472,Q4,Makanan untuk penderita penyakit lambung dan l...,makanan penderita penyakit lambung liver dok m...
624,Q5,Kebiasaan melamun atau berkhayal secara berleb...,kebiasaan melamun berkhayal dok pengidap malad...


In [85]:
queries = queries.rename(columns = {"query": "query_raw", "query_preprocessed": "query"})
queries.head()

,qid,query_raw,query
0,Q1,"Cara mengatasi pusing, lemas dan hidung berair...",mengatasi pusing lemas hidung berair malam dok...
174,Q2,Telinga berdengung sebelah kiri. Selamat pagi ...,telinga berdengung sebelah kiri selamat pagi d...
308,Q3,Benjolan di sekitar kelamin. Sore dok..Maaf do...,benjolan kelamin sore dok maaf dok 4 mengalami...
472,Q4,Makanan untuk penderita penyakit lambung dan l...,makanan penderita penyakit lambung liver dok m...
624,Q5,Kebiasaan melamun atau berkhayal secara berleb...,kebiasaan melamun berkhayal dok pengidap malad...


## Index

In [86]:
%%time
# input for PyTerrier indexer must be a dataframe containing "docno" and "text"

collection = data.rename(columns = {"keluhan_raw": "text"})

!rm -rf ./medical_ir_index
pd_indexer = pt.DFIndexer("./medical_ir_index", \
                          type = pt.index.IndexingType(1), \
                          tokeniser = "UTFTokeniser", \
                          stemmer = None, \
                          stopwords = None, \
                          blocks = True)
index_ref = pd_indexer.index(collection["keluhan"], collection)

collection.info()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


04:29:45.568 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (KD-20960) - further warnings are suppressed
04:30:03.142 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents
<class 'pandas.core.frame.DataFrame'>
Index: 86723 entries, 0 to 86730
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   docno    86723 non-null  object
 1   text     86723 non-null  object
 2   keluhan  86723 non-null  object
dtypes: object(3)
memory usage: 2.6+ MB
CPU times: user 1min 6s, sys: 676 ms, total: 1min 6s
Wall time: 1min 5s


In [87]:
index_fact = pt.IndexFactory.of(index_ref)
print(index_fact.getCollectionStatistics().toString())

Number of documents: 86723
Number of terms: 74200
Number of postings: 1924462
Number of fields: 0
Number of tokens: 2575525
Field names: []
Positions:   true



# Re-rank

In [88]:
CUT_OFF = 15

#### Features Creation

#### Cross Validation

In [89]:
""" Model retrieval """

bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25")
bb2 = pt.BatchRetrieve(index_ref, wmodel = "BB2")
tf = pt.BatchRetrieve(index_ref, wmodel = "Tf")
tf_idf = pt.BatchRetrieve(index_ref, wmodel = "TF_IDF")
pl2 = pt.BatchRetrieve(index_ref, wmodel = "PL2")
cm = pt.BatchRetrieve(index_ref, wmodel = "CoordinateMatch")
dfic = pt.BatchRetrieve(index_ref, wmodel = "DFIC")
dfiz = pt.BatchRetrieve(index_ref, wmodel = "DFIZ")
dfr_bm25 = pt.BatchRetrieve(index_ref, wmodel = "DFR_BM25")
dfree = pt.BatchRetrieve(index_ref, wmodel = "DFRee")
dfreeklim = pt.BatchRetrieve(index_ref, wmodel = "DFReeKLIM")
dirichlet = pt.BatchRetrieve(index_ref, wmodel = "DirichletLM")
dl = pt.BatchRetrieve(index_ref, wmodel = "Dl")
dlh = pt.BatchRetrieve(index_ref, wmodel = "DLH")
dlh13 = pt.BatchRetrieve(index_ref, wmodel = "DLH13")
dph = pt.BatchRetrieve(index_ref, wmodel = "DPH")
hiemstra = pt.BatchRetrieve(index_ref, wmodel = "Hiemstra_LM")
ifb2 = pt.BatchRetrieve(index_ref, wmodel = "IFB2")
in_expb2 = pt.BatchRetrieve(index_ref, wmodel = "In_expB2")
in_expc2 = pt.BatchRetrieve(index_ref, wmodel = "In_expC2")
inb2 = pt.BatchRetrieve(index_ref, wmodel = "InB2")
inl2 = pt.BatchRetrieve(index_ref, wmodel = "InL2")
js_kls = pt.BatchRetrieve(index_ref, wmodel = "Js_KLs")
lemurtf_idf = pt.BatchRetrieve(index_ref, wmodel = "LemurTF_IDF")
lgd = pt.BatchRetrieve(index_ref, wmodel = "LGD")
null = pt.BatchRetrieve(index_ref, wmodel = "Null")
xsqra_m = pt.BatchRetrieve(index_ref, wmodel = "XSqrA_M")

In [90]:
crossmodel = CrossEncoder('cross-encoder/quora-roberta-base', max_length = 512, device='cuda:0')

def _crossencoder_apply(df : pd.DataFrame):
    return crossmodel.predict(list(zip(df['query_raw'].values, df['text'].values)))

cross_encT = pt.apply.doc_score(_crossencoder_apply, batch_size=128)

In [91]:
# feature adalah fungsi dari pasangan dokumen
# dan query --> feature(dokumen, query)
def avg_vector_embedding(model_embedding, tokenized_data):
    """
    Representasi sederhana dokumen sebagai rata-rata vektor
    hasil word embedding dari seluruh kata yang ada di dokumen tersebut
    """
    embedding = []
    vector_acc = np.zeros(VECTOR_SIZE)
    count = 0
    for word in tokenized_data:
        if word in model_embedding.wv:
            vector_acc += model_embedding.wv[word]
            count += 1
    if count > 0:
        vector_acc /= count
    embedding.append(vector_acc)
    return np.array(embedding)

def f_len_diff(text, query):
    """ Delta panjang dokumen & query """
    return abs(len(text.split()) - len(query.split()))

def f_jaccard(text, query):
    """ Jaccard Similarity """
    word_set_text = set(text)
    word_set_query = set(query)
    return len(word_set_text & word_set_query) / len(word_set_text | word_set_query)

def f_canberra(text_embedding, query_embedding):
    """ Canberra Distance """
    return distance.canberra(text_embedding, query_embedding)

def f_cosine(text_embedding, query_embedding):
    """ Cosine Similarity """
    return cosine(text_embedding, query_embedding)

def f_wmd(model, text, query):
    """ Word Mover Distance """
    word_text = text.split()
    word_query = query.split()
    MAX_DISTANCE_VALUE = 10.0
    if all(token in model.wv for token in word_text) and all(token in model.wv for token in word_query):
        distance = model.wv.wmdistance(word_text, word_query)
    else:
        distance = MAX_DISTANCE_VALUE

#     word_text = [token for token in text.split() if token in model.wv.index_to_key]
#     word_query = [token for token in query.split() if token in model.wv.index_to_key]
#     MAX_DISTANCE_VALUE = 10.0
#     if not word_text or not word_query:
#         distance = MAX_DISTANCE_VALUE
#     else:
#         distance = model.wv.wmdistance(word_text, word_query)
    return distance

def f_common_distinct_words(text, query):
    word_set_text = set(text.split())
    word_set_query = set(query.split())
    return len(word_set_text & word_set_query)

def f_common_context_nowindow(text, query):
    r = Rake(stops)
    r.extract_keywords_from_text(query)
    query_context = r.get_ranked_phrases()
    context_words = set()
    for phrase in query_context:
        words = set(phrase.split())
        for word in words:
            context_words.add(word)
    return len(set(text.split()) & context_words) / len(set(text.split()) | context_words)

def f_common_context_soft_nowindow(text, query):
    r = Rake(stops)
    r.extract_keywords_from_text(query)
    query_context = r.get_ranked_phrases()
    context_words = []
    for phrase in query_context:
        words = set(phrase.split())
        for word in words:
            context_words.append(word)
    return f_wmd(model_ft, text, " ".join(context_words))

def f_common_context_soft(text, query, window=10):
    earliest_words = query.split()[:window]
    latest_words = query.split()[-window:]

    r = Rake(stops)
    r.extract_keywords_from_text(" ".join(earliest_words))
    earliest_context = r.get_ranked_phrases()

    r = Rake(stops)
    r.extract_keywords_from_text(" ".join(latest_words))
    latest_context = r.get_ranked_phrases()

    query_context = set(earliest_context + latest_context)
    context_words = []
    for phrase in query_context:
        words = set(phrase.split())
        for word in words:
            context_words.append(word)

    return f_wmd(model_ft, text, " ".join(context_words))

def f_common_context(text, query, window=10):
    earliest_words = query.split()[:window]
    latest_words = query.split()[-window:]

    r = Rake(stops)
    r.extract_keywords_from_text(" ".join(earliest_words))
    earliest_context = r.get_ranked_phrases()

    r = Rake(stops)
    r.extract_keywords_from_text(" ".join(latest_words))
    latest_context = r.get_ranked_phrases()

    query_context = set(earliest_context + latest_context)
    context_words = set()
    for phrase in query_context:
        words = set(phrase.split())
        for word in words:
            context_words.add(word)
    
    return len(set(text.split()) & context_words) / len(set(text.split()) | context_words)

def f_common_context_2(text, query, window=10):
    earliest_words = query.split()[:window]
    latest_words = query.split()[-window:]
    r = Rake(stops)
    r.extract_keywords_from_text(" ".join(earliest_words))
    earliest_context = r.get_ranked_phrases()
    r = Rake(stops)
    r.extract_keywords_from_text(" ".join(latest_words))
    latest_context = r.get_ranked_phrases()

    query_context = set(earliest_context + latest_context)
    
    earliest_words = text.split()[:window]
    latest_words = text.split()[-window:]
    r = Rake(stops)
    r.extract_keywords_from_text(" ".join(earliest_words))
    earliest_context = r.get_ranked_phrases()
    r = Rake(stops)
    r.extract_keywords_from_text(" ".join(latest_words))
    latest_context = r.get_ranked_phrases()

    text_context = set(earliest_context + latest_context)
    
    return len(text_context & query_context) / len(text_context | query_context)


def f_common_distinct_words_window(text, query, window=10):
    context_words = set(query.split()[:window] + query.split()[-window:])
    return len(set(text.split()) & set(context_words)) / len(set(text.split()) | set(context_words))

def combine_f(row):
    docno = row["docno"]
    qid = row["qid"]
    keluhan = row["keluhan"]
    query = row["query"]
    keluhan_raw = row["text"]
    query_raw = row["query_raw"]
    
    text_w2v_embedding = avg_vector_embedding(model_w2v, keluhan.split())
    query_w2v_embedding = avg_vector_embedding(model_w2v, query.split())
    text_ft_embedding = avg_vector_embedding(model_ft, keluhan.split())
    query_ft_embedding = avg_vector_embedding(model_ft, query.split())
    
    return np.array([
                    # f_len_diff(keluhan, query), \
                    # f_jaccard(keluhan.split(), query.split()), \
        
                    # f_cosine(text_w2v_embedding.flatten(), query_w2v_embedding.flatten()), \
                    f_cosine(text_ft_embedding.flatten(), query_ft_embedding.flatten()), \
        
                    # f_cosine(pminilm_doc_embeddings[docno], pminilm_query_embeddings[qid]), \
                    f_cosine(t5_doc_embeddings[docno], t5_query_embeddings[qid]), \
                    f_cosine(bert_doc_embeddings[docno], bert_query_embeddings[qid]), \
                    
                    # f_wmd(model_w2v, keluhan, query), \
                    # f_wmd(model_ft, keluhan, query), \

                    f_canberra(text_w2v_embedding.flatten(), query_w2v_embedding.flatten()), \
                    # f_canberra(text_ft_embedding.flatten(), query_ft_embedding.flatten()), \

                    # f_common_distinct_words(keluhan, query), \
                    # f_common_distinct_words(keluhan_raw, query_raw), \
                    
                    f_common_context(keluhan_raw, query_raw), \
                    f_common_context_soft(keluhan_raw, query_raw), \
                    # f_common_context_nowindow(keluhan_raw, query_raw), \
                    # f_common_context_soft_nowindow(keluhan_raw, query_raw), \
                    # f_common_context_2(keluhan_raw, query_raw), \
                    # f_common_distinct_words_window(keluhan_raw, query)
                   ])

In [92]:
%%time

""" Model word embedding """
tokenized_corpus = []
for doc in data['keluhan']:
    tokenized_corpus.append(doc.split())
VECTOR_SIZE = 64 
model_w2v = Word2Vec(tokenized_corpus, vector_size=VECTOR_SIZE, min_count=2, workers=1) # CBOW
model_ft = FastText(tokenized_corpus, vector_size=VECTOR_SIZE, min_count=2, workers=1)

""" KFold """
FOLD = 5
kf = KFold(n_splits = FOLD, shuffle=True, random_state=42)

evaluation_results = pd.DataFrame()
metrics = dict()
metric_names = [P@10, P@5, "map", "recip_rank", nDCG@5]
model_names = ["BM25 Only", "BM25 >> LambdaMART XGBoost"]

""" Cross validation """
for train_idx, test_idx in kf.split(queries):
    train_queries, test_queries = queries.iloc[train_idx], queries.iloc[test_idx]
    
    print(f"TRAIN DATA: {len(qrels[qrels['qid'].isin(train_queries['qid'].values)])}")
    print(f"TEST DATA: {len(qrels[qrels['qid'].isin(test_queries['qid'].values)])}")

    more_features = pt.apply.doc_features(lambda row: combine_f(row))
    pipeline = (bm25 % CUT_OFF) >> pt.text.get_text(index_ref, ["text", "keluhan"]) >> (more_features ** bm25 ** cross_encT)

    lmart_xgb = xgb.sklearn.XGBRanker(objective = 'rank:ndcg',
                                  learning_rate = 0.1,
                                  gamma = 1.0,
                                  min_child_weight = 0.1,
                                  max_depth = 6,
                                  random_state = 42,
                                  tree_method='hist',
                                  device="cuda"
                            )

    lmart_xgb_pipeline = pipeline >> pt.ltr.apply_learned_model(lmart_xgb, form = "ltr")
    lmart_xgb_pipeline.fit(train_queries, qrels, test_queries, qrels)

    evaluation_results = pd.concat([evaluation_results, \
                                  pt.Experiment([(bm25 % CUT_OFF), lmart_xgb_pipeline], \
                                                test_queries, \
                                                qrels, \
                                                eval_metrics=metric_names, \
                                                names=model_names, \
                                                perquery=True)], \
                                  ignore_index=True)
    
""" Konversi """
for index, line in evaluation_results.iterrows():
    model_name = line['name']
    qid = line['qid']
    measure = line['measure']
    score = line['value']

    if measure in metrics:
        if model_name in metrics[measure]:
            metrics[measure][model_name][qid] = score
        else:
            metrics[measure][model_name] = {qid: score}
    else:
        metrics[measure] = {model_name: {qid: score}}

""" Metrik """
for metric_name in metrics:
    df_metric = pd.DataFrame(metrics[metric_name])
    baseline = df_metric['BM25 Only']
    rerank = df_metric['BM25 >> LambdaMART XGBoost']

    print(f"Rata-rata {metric_name} untuk baseline: {np.mean(baseline)}")
    print(f"Rata-rata {metric_name} untuk model rerank: {np.mean(rerank)}")
    print(stats.ttest_rel(baseline, rerank))

    print()

TRAIN DATA: 4910
TEST DATA: 1200


pt.apply:   0%|          | 0/5 [00:00<?, ?row/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

pt.apply:  20%|██        | 1/5 [00:03<00:14,  3.66s/row]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

pt.apply:  50%|█████     | 1/2 [00:02<00:02,  2.28s/row]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

pt.apply: 100%|██████████| 2/2 [00:03<00:00,  1.98s/row]

Rata-rata map untuk baseline: 0.19187186327527717
Rata-rata map untuk model rerank: 0.19509580829482742
TtestResult(statistic=-0.924442647851911, pvalue=0.3602982554191462, df=44)

Rata-rata recip_rank untuk baseline: 0.8577777777777776
Rata-rata recip_rank untuk model rerank: 0.9203703703703705
TtestResult(statistic=-1.4681353710410059, pvalue=0.14918042779229887, df=44)

Rata-rata P@5 untuk baseline: 0.7377777777777776
Rata-rata P@5 untuk model rerank: 0.7155555555555556
TtestResult(statistic=0.7973946013668627, pvalue=0.42950418844635174, df=44)

Rata-rata P@10 untuk baseline: 0.6555555555555556
Rata-rata P@10 untuk model rerank: 0.6577777777777777
TtestResult(statistic=-0.15447061651522734, pvalue=0.877945064167393, df=44)

Rata-rata nDCG@5 untuk baseline: 0.7448905273418782
Rata-rata nDCG@5 untuk model rerank: 0.7424661925264584
TtestResult(statistic=0.09608536898090199, pvalue=0.9238890792746858, df=44)

CPU times: user 5min 54s, sys: 1.53 s, total: 5min 55s
Wall time: 5min 50s


In [93]:
lmart_xgb.get_booster().get_score(importance_type='gain')

{'f0': 2.360785484313965,
 'f1': 1.8442738056182861,
 'f2': 2.0295732021331787,
 'f3': 2.5701160430908203,
 'f4': 1.9449294805526733,
 'f5': 2.3094310760498047,
 'f6': 2.3934743404388428,
 'f7': 1.581977128982544}

In [94]:
lmart_xgb.get_booster().get_score(importance_type='weight')

{'f0': 20.0,
 'f1': 13.0,
 'f2': 13.0,
 'f3': 35.0,
 'f4': 15.0,
 'f5': 16.0,
 'f6': 38.0,
 'f7': 17.0}

In [95]:
lmart_xgb.feature_importances_

array([0.13858798, 0.10826658, 0.11914444, 0.15087657, 0.1141755 ,
       0.13557327, 0.14050695, 0.09286868], dtype=float32)

# Analyze

In [96]:
def get_serp_metadata(serp):
    return pd.merge(serp, qrels[['qid', 'docno', 'label']], how='left', on=['qid', 'docno'])

def get_relevant_serp(serp):
    relevant_docno = pd.merge(serp, qrels[(qrels['label'] == 1)], how='inner', on='docno')['docno'].unique()
    return serp[serp['docno'].isin(relevant_docno)]

def compare_rank_score(metric_scores, test_topics):
    df_scores = pd.DataFrame(metric_scores)
    df_scores = df_scores[df_scores.index == test_topics['qid']]

    observed_queries = []
    for index, row in df_scores.iterrows():
        observed_queries.append(queries[queries['qid'] == index]['query_raw'].values[0])
    df_scores['query_raw'] = observed_queries

    print("Rerank < Baseline (Baseline > Rerank)")
    display(df_scores[df_scores['BM25 >> LambdaMART XGBoost'] < df_scores['BM25 Only']])
    print()
    print("Rerank > Baseline (Baseline < Rerank)")
    display(df_scores[df_scores['BM25 >> LambdaMART XGBoost'] > df_scores['BM25 Only']])

def get_observed_doc(observed_docno):
    return data[data['docno'] == observed_docno]['keluhan_raw'].values[0]
    
def get_observed_query(observed_qid):
    return queries[queries['qid'] == observed_qid]['query_raw'].values[0]

def display_observed_query(observed_qid):
    print(f"{observed_qid}: {get_observed_query(observed_qid)}")

def get_serp(observed_qid, model, get_qrels_label=True, k=15):
    model_result = model.transform(queries[queries['qid'] == observed_qid])
    if get_qrels_label:
        model_result = get_serp_metadata(model_result)
    top_metadata = model_result[model_result['rank'] < k].sort_values(by=['rank'])
    return top_metadata
    
def display_serp(observed_qid, model, get_qrels_label=True, k=15):
    display(get_serp(observed_qid, model, get_qrels_label, k))

In [97]:
from sklearn.model_selection import train_test_split
train_topics, test_topics = train_test_split(queries, test_size=0.2, random_state=2)
test_topics

,qid,query_raw,query
5580,Q47,"Mental Disorder atau tidak ?. Salam, Saya ada ...",mental disorder salam pendekatan gadis cantik ...
2442,Q20,Serangan panik dan sulit bernafas. Malam dokte...,serangan panik sulit bernafas malam dokter men...
5765,Q49,Apakah pusing merupakan efek dari sakit flu. S...,pusing efek sakit flu selamat sore dok irsan 2...
1951,Q16,Mengatasi Gangguan Penglihatan Usai Kena Belek...,mengatasi gangguan penglihatan kena belekan do...
5941,Q50,"Sariawan Dan Kaitannya Dengan Demam, Mual, Pus...",sariawan kaitannya demam mual pusing dok 1 min...
4058,Q34,"Mata Silinder. Dok saya mau nanya, awalnya mat...",mata silinder dok nanya mata minus trs kemarin...
1673,Q14,"Pakai Aloe Vera Malah Bikin Berjerawat, Mengap...",pakai aloe vera bikin berjerawat sore dok prod...
1823,Q15,Berbahayakah Makan Mi Instan Setiap Hari?. Dok...,berbahayakah makan mi instan dok berbahaya ngg...
0,Q1,"Cara mengatasi pusing, lemas dan hidung berair...",mengatasi pusing lemas hidung berair malam dok...


In [98]:
%%time

# UNCOMMENT FOR USE

tokenized_corpus = []
for doc in data['keluhan']:
    tokenized_corpus.append(doc.split())
VECTOR_SIZE = 64
model_w2v = Word2Vec(tokenized_corpus, vector_size=VECTOR_SIZE, min_count=2, workers=1) # CBOW
model_ft = FastText(tokenized_corpus, vector_size=VECTOR_SIZE, min_count=2, workers=1)

metric_names = [P@10, P@5, "map", "recip_rank", nDCG@5]
model_names = ["BM25 Only", "BM25 >> LambdaMART XGBoost"]

more_features = pt.apply.doc_features(lambda row: combine_f(row))
pipeline = (bm25 % CUT_OFF) >> pt.text.get_text(index_ref, ["text", "keluhan"]) >> (more_features ** bm25 ** cross_encT)

lmart_xgb2 = xgb.sklearn.XGBRanker(objective = 'rank:ndcg',
                              learning_rate = 0.1,
                              gamma = 1.0,
                              min_child_weight = 0.1,
                              max_depth = 6,
                              random_state = 42,
                              tree_method='hist',
                              device="cuda"
                        )

lmart_xgb_pipeline2 = pipeline >> pt.ltr.apply_learned_model(lmart_xgb2, form = "ltr")
lmart_xgb_pipeline2.fit(train_topics, qrels, test_topics, qrels)

evaluation_results2 = pt.Experiment([bm25 % CUT_OFF, lmart_xgb_pipeline2], \
                                          test_topics, \
                                          qrels, \
                                          eval_metrics=metric_names, \
                                          names=model_names, \
                                          perquery=True)

""" Konversi """
metrics2 = dict()
for index, line in evaluation_results2.iterrows():
    model_name = line['name']
    qid = line['qid']
    measure = line['measure']
    score = line['value']

    if measure in metrics2:
        if model_name in metrics2[measure]:
            metrics2[measure][model_name][qid] = score
        else:
            metrics2[measure][model_name] = {qid: score}
    else:
        metrics2[measure] = {model_name: {qid: score}}

pt.apply:   0%|          | 0/5 [00:00<?, ?row/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

pt.apply:  20%|██        | 1/5 [00:03<00:14,  3.67s/row]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

pt.apply:  40%|████      | 2/5 [00:07<00:10,  3.53s/row]

...

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

pt.apply:  50%|█████     | 1/2 [00:02<00:02,  2.41s/row]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

pt.apply: 100%|██████████| 2/2 [00:04<00:00,  2.15s/row]

CPU times: user 2min 24s, sys: 506 ms, total: 2min 24s
Wall time: 2min 23s


In [99]:
metrics2['recip_rank']

{'BM25 >> LambdaMART XGBoost': {'Q1': 1.0,
  'Q14': 1.0,
  'Q15': 1.0,
  'Q16': 1.0,
  'Q20': 1.0,
  'Q34': 1.0,
  'Q47': 0.14285714285714285,
  'Q49': 1.0,
  'Q50': 1.0},
 'BM25 Only': {'Q1': 1.0,
  'Q14': 0.25,
  'Q15': 1.0,
  'Q16': 1.0,
  'Q20': 0.3333333333333333,
  'Q34': 1.0,
  'Q47': 0.25,
  'Q49': 1.0,
  'Q50': 1.0}}

In [100]:
compare_rank_score(metrics2['recip_rank'], test_topics)

Rerank < Baseline (Baseline > Rerank)


,BM25 >> LambdaMART XGBoost,BM25 Only,query_raw



Rerank > Baseline (Baseline < Rerank)


,BM25 >> LambdaMART XGBoost,BM25 Only,query_raw


In [101]:
def compare_rank_score_all(metric_scores):
    df_scores = pd.DataFrame(metric_scores)

    observed_queries = []
    for index, row in df_scores.iterrows():
        observed_queries.append(queries[queries['qid'] == index]['query_raw'].values[0])
    df_scores['query_raw'] = observed_queries

    print("Rerank < Baseline (Baseline > Rerank)")
    display(df_scores[df_scores['BM25 >> LambdaMART XGBoost'] < df_scores['BM25 Only']])
    print()
    print("Rerank > Baseline (Baseline < Rerank)")
    display(df_scores[df_scores['BM25 >> LambdaMART XGBoost'] > df_scores['BM25 Only']])

compare_rank_score_all(metrics2['recip_rank'])

Rerank < Baseline (Baseline > Rerank)


,BM25 >> LambdaMART XGBoost,BM25 Only,query_raw
Q47,0.142857,0.25,"Mental Disorder atau tidak ?. Salam, Saya ada ..."



Rerank > Baseline (Baseline < Rerank)


,BM25 >> LambdaMART XGBoost,BM25 Only,query_raw
Q14,1.0,0.250000,"Pakai Aloe Vera Malah Bikin Berjerawat, Mengap..."
Q20,1.0,0.333333,Serangan panik dan sulit bernafas. Malam dokte...


In [102]:
serp = get_serp('Q47', lmart_xgb_pipeline2)
with open('serp_Q47_hard&soft_REV.pickle', 'wb') as handle:
    pickle.dump(serp, handle, protocol=pickle.HIGHEST_PROTOCOL)

pt.apply:   0%|          | 0/1 [00:00<?, ?row/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

pt.apply: 100%|██████████| 1/1 [00:00<00:00,  2.17row/s]
